In [6]:
import pandas as pd
import xgboost as xgb
from xgboost import XGBClassifier
import datetime; import pytz
import matplotlib as plt
from scipy.special import softmax
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split # (*arrays, **options)
import numpy as np
from sklearn.metrics import log_loss
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

from joblib import dump, load
import joblib

from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.model_selection import KFold, train_test_split, GridSearchCV

In [2]:
# This time, I am going to build the dataset balanced out to start with
# 

In [3]:
datadir = '/opt/data'
localdir = '/opt/program'
tripsdf = pd.read_csv(f'{datadir}/2013-07 - Citi Bike trip data.csv')
stationsdf = pd.read_csv(f'{localdir}/datas/stations/stations-2018-12-04-c.csv',
                        index_col=0)

In [4]:
def prepare_data(tripsdf, stationsdf):
    
    # Step 1, merge w/ stationsdf to get neighborhood data
    mdf = tripsdf[['start station name', 'end station name', 'gender']
            ].merge(stationsdf[['station_name', 'neighborhood']], 
                    left_on='start station name',
                    right_on='station_name'
                   ).rename(columns={'neighborhood': 'start_neighborhood'}
                           ).merge(stationsdf[['station_name', 'neighborhood']],
                                  left_on='end station name',
                                   right_on='station_name'
                                  ).rename(columns={'neighborhood': 'end_neighborhood'})
    
    neighborhoods = sorted(stationsdf.neighborhood.unique().tolist())
    
    X, y = (mdf[['start_neighborhood', 'gender']].values, 
            np.array(mdf['end_neighborhood'].tolist()))
    return X, y
    
def preprocess(X, y, neighborhoods):
    # Initially assuming labeled=True
    labeled = True
    genders = [0, 1, 2]

    enc = OneHotEncoder(handle_unknown='error', 
                        categories=[neighborhoods, genders])
    enc.fit(X)
    X_transformed = enc.transform(X)
    
    le = LabelEncoder()
    le.fit(neighborhoods)
    
    y_enc = le.transform(y)    
    
    return X_transformed, enc, le, y_enc
    
class FooFlassifier():
    def __init__(self, stationsdf):
        self.stationsdf = stationsdf
        self.neighborhoods = sorted(stationsdf.neighborhood.unique().tolist())
        self.workdir = make_work_dir()
    def fit(self, X, y):
        # preproc
        (X_transformed,
             self.one_hot_enc, self.le,
             y_enc) = preprocess(X, y, self.neighborhoods)
        joblib.dump({'le': self.le,
                    'one_hot_enc': self.one_hot_enc},
                   f'{self.workdir}/artifacts.joblib')
        self.clf = XGBClassifier()
        self.clf.fit(X_transformed, y_enc)
        
        dtrain = xgb.DMatrix(X_transformed, label=y_enc)
        params = {'max_depth':2, 'eta':1, 'objective': 'multi:softprob'}
      
        
        self.labels = list(range(len(self.le.classes_)))
        
#    def score(self, X, y_true):
#        X_transformed = self.one_hot_enc.transform(X)
#        y_true_enc = self.le.transform(y_true)
        
#        y_prob = self.clf.predict_proba(X_transformed)
#        return log_loss(y_true_enc, y_prob, labels=self.labels)

    def get_params(self, deep):
        return {}
    
def make_work_dir():
    ts = utc_ts()
    workdir = f'/opt/program/artifacts/{ts}' 
    os.mkdir(workdir)
    return workdir

#X, y = mdf[['start_neighborhood', 'gender']].values, np.array(mdf['end_neighborhood'].tolist())

#clf = FooFlassifier(stationsdf) 

In [7]:
%%time
rng = np.random.RandomState(31337)
X, y = prepare_data(tripsdf, stationsdf)
neighborhoods = sorted(stationsdf.neighborhood.unique().tolist())


kf = KFold(n_splits=2, shuffle=True, random_state=rng)
for train_index, test_index in kf.split(X):
    
    # preproc
    (X_transformed,
         one_hot_enc, le,
         y_enc) = preprocess(X[train_index], y[train_index], 
                             neighborhoods)
    
    xgb_model = xgb.XGBClassifier().fit(X_transformed, 
                                        y_enc)
    #
    X_test_transformed = one_hot_enc.transform(X[test_index])
    y_true_enc = le.transform(y[test_index])
    
    predictions = xgb_model.predict(X_test_transformed)
    actuals = y_true_enc
    print(confusion_matrix(actuals, predictions))
    

[[   0   49   12 ...    0    0  143]
 [   0 1112   13 ...    0    0   39]
 [   0    1  283 ...    0    0  305]
 ...
 [   0    0    7 ...    0    0   73]
 [   0  567   27 ...    0    0   57]
 [   0   21  163 ...    0    0 2064]]
[[ 193   64   14 ...    0    0  150]
 [  27 1167   16 ...    0    0   28]
 [   4    6  379 ...    0    0  236]
 ...
 [   0    0   17 ...    0    0   46]
 [  67  599   24 ...    0    0   78]
 [  19    9  196 ...    0    0 2017]]
CPU times: user 19min 36s, sys: 2.99 s, total: 19min 39s
Wall time: 19min 45s


In [21]:
joblib.dump({'notebook': '2020-06-10-again',
            'model': xgb_model,
            'actuals': actuals,
            'predictions': predictions,
             'confusion_matrix': confusion_matrix(actuals, predictions),
             'walltime_train': '19min 45s',
             'preproc': {'le': le, 'one_hot_enc': one_hot_enc}
            }, f'{workdir}/bundle.joblib')

['/opt/program/artifacts/2020-06-11T041641Z/bundle.joblib']

In [15]:

def utc_ts():
    return datetime.datetime.utcnow(
        ).replace(tzinfo=pytz.UTC).strftime('%Y-%m-%dT%H%M%SZ')
utc_ts()

'2020-06-11T041623Z'

In [18]:
import os
workdir = make_work_dir()
pd.DataFrame.from_records(confusion_matrix(actuals, predictions),
                          columns=range(54)).to_csv(f'{workdir}/confusion.csv')

In [16]:

def fit(X, y):
    # do preproc...
    X_trans <== one hot enc(X)
    y_enc <== y

dtrain = xgb.DMatrix(X_trans[:1000], label=y_enc[:1000])
params = {}
num_round = 2
xgb.cv(
    params,
    dtrain,
    num_round,
    nfold=5,
    metrics={'mlogloss', 'error'},
    callbacks=[xgb.callback.print_evaluation(show_stdv=True)]
    )
model = FooFlassifier(stationsdf)
ipdb.runcall(model.fit, X[:100, :], y[:100])


SyntaxError: invalid syntax (<ipython-input-16-6897dd5d4cdb>, line 3)